# **1. Import TensorFlow and other libraries**

In [1]:
pip install -q sklearn

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Conv2D, MaxPool2D, Flatten
import keras

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# **2. Use Pandas to create a dataframe**

In [3]:
import pathlib

dataset_url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/21795/1405944/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1597545112&Signature=m%2FzXgrtu1%2B1voyfUg3zXWtLaDjYrXPpB0b4pRF2BwcxDThVUJEC51HmelLWF67BZ61YFKLIZ7pFn752NRFiX7WrV0J4azENRAA%2BAvrzJLrXrxVQ5S34K6Ul3OA3jQ4GSM8qm6bKLjzyfoOE46pk%2FaJyigryh1cIjEgxgw6pLqZ7uKd%2B3bQZ11J5ycjTKDUjHTKyh2YZ%2BodjLo7r8u95fUnODnZnhuQOOBetgijndj4R1aJ45wE3S6O%2FS3ghmjo4ri3Vj4O4CZRNmhu550q1Kxh%2BocjhZC7lgJMDrJxYXu6CGHtRF0VzjoNPZ5OzgK9UPfwEQ9D%2F8cy%2BKcRKPgjGUcg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip'
csv_file = 'datasets/train.csv'

tf.keras.utils.get_file('train.csv.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

99229696/99224042 [==============================] - 1s 0us/step


In [4]:
dataframe.head()

,# Id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f986,f987,f988,f989,f990,f991,f992,f993,f994,f995,f996,f997,f998,f999,f1000,f1001,f1002,f1003,f1004,f1005,f1006,f1007,f1008,f1009,f1010,f1011,f1012,f1013,f1014,f1015,f1016,f1017,f1018,f1019,f1020,f1021,f1022,f1023,f1024,Category
0,1,0.685075,0.000000,2.159684,0.000000,0.000000,1.855760,0.985496,0.000000,0.000000,0.000000,0.166518,0.000000,0.0,0.927346,0.590299,0.364779,0.721798,0.333858,1.218988,0.817693,0.379442,0.000000,1.369456,0.000000,0.0,0.000000,0.0,0.0,1.152172,1.427282,0.000000,1.376098,0.478925,0.000000,0.605684,1.001201,0.000000,0.293933,2.312609,...,0.000000,2.970812,0.000000,0.000000,0.035363,1.251016,1.155578,0.299259,0.863717,1.448987,0.085982,1.350596,0.000000,0.407195,3.773583,1.022316,0.060898,0.000000,0.695031,0.000000,1.060223,0.860375,0.442289,0.163974,0.543900,1.767029,0.702262,3.539757,0.941511,1.570190,0.000000,0.687147,1.416029,0.398683,1.203272,0.833575,2.119920,0.266268,0.000000,8
1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.176542,0.704747,0.000000,0.192226,0.868009,0.608963,1.028225,0.0,1.481206,0.583469,0.184480,0.000000,0.511936,0.000000,0.000000,0.000000,1.276256,0.534841,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.887524,1.904664,0.000000,0.791537,0.000000,1.288954,0.075975,0.432621,0.000000,0.809392,...,1.002027,2.302625,0.279369,0.009015,0.630017,0.000000,0.798314,1.415711,1.010964,1.607744,0.014198,2.020489,0.000000,0.612987,3.451463,1.602385,0.030343,0.000000,0.059625,0.000000,1.066282,1.360797,0.559455,0.327810,1.203236,1.195106,1.137062,3.593266,2.300208,1.768872,0.000000,0.935894,0.508511,0.267381,1.249635,1.641100,2.148436,0.582175,0.078609,3
2,3,2.002489,0.000000,0.978972,0.000000,0.327216,0.000000,0.454218,0.305732,0.000000,1.249242,1.773079,0.266320,0.0,0.932390,0.888442,1.170573,0.500965,0.324251,0.749860,0.000000,1.150037,1.779132,0.771711,0.004418,0.0,3.358535,0.0,0.0,0.000000,1.359771,1.357755,1.757337,2.039395,0.555814,2.429130,0.000000,0.000000,0.627085,0.441804,...,0.000000,0.559122,0.277996,0.000000,0.000000,0.108532,0.693938,0.317077,0.000000,1.464030,0.201412,0.739319,0.123736,0.055401,0.827199,0.073738,0.000000,0.000000,0.000000,0.158605,0.043113,0.000000,0.595746,0.000000,0.000000,1.070579,1.061942,1.288309,1.183666,0.000000,0.498108,0.918560,1.056273,0.508074,0.737373,0.726668,1.169889,0.000000,0.000000,7
3,4,0.722091,0.000000,2.304015,0.000000,0.000000,1.823341,0.721335,0.000000,0.166690,0.246660,0.389565,0.000000,0.0,0.865144,0.000000,1.067876,1.176675,0.371711,1.006336,0.000000,0.000000,0.000000,0.289524,0.000000,0.0,0.000000,0.0,0.0,2.468661,2.136180,0.000000,0.958361,0.402866,0.000000,1.683738,0.555321,0.000000,0.935477,1.333827,...,1.742886,1.880181,0.568967,0.000000,1.000142,0.319556,0.000000,0.269205,0.913789,0.000000,0.000000,2.044642,2.088821,0.153658,2.469944,1.959049,0.570214,0.530098,0.000000,0.000000,0.237062,0.000000,1.301427,0.000000,0.000000,1.636730,0.359067,2.053269,0.000000,2.078060,1.587578,0.365814,0.000000,0.747819,0.074074,0.000000,0.520719,0.000000,0.931651,1
4,5,0.000000,0.068588,1.038505,0.097392,0.000000,0.598382,1.102372,0.000000,0.000000,0.682103,0.546078,0.000000,0.0,0.969638,0.000000,0.000000,0.000000,0.763285,0.008783,0.000000,0.000000,0.619380,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.626301,0.299262,0.588616,0.421499,0.651188,0.000000,1.074915,0.000000,0.387621,0.000000,1.230561,...,0.000000,0.509055,0.000000,0.000000,0.000000,0.488458,0.401302,0.572882,0.000000,1.206674,0.607579,1.473128,0.013943,0.153198,0.927134,0.354164,0.000000,0.143861,0.000000,0.215866,0.000000,0.000000,0.111050,0.104496,0.000000,0.504245,1.066074,0.793241,1.627079,0.863230,0.071165,0.000000,1.121873,0.778090,0.334674,0.583489,0.428917,0.000000,0.000000,1


#**3. Create target variable**

In [5]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = dataframe['Category']

# Drop un-used columns.
dataframe = dataframe.drop(columns=['Category', '# Id'])

#**4. Split the dataframe into train, validation, and test**

In [6]:
# train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(dataframe, test_size=0.25)
print(len(train), 'train examples')
print(len(val), 'validation examples')
# print(len(test), 'test examples')

22500 train examples
7500 validation examples


#**5. Create an input pipeline using tf.data**

In [7]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [11]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of f1:', feature_batch['f1'])
  print('A batch of targets:', label_batch )

Every feature: ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f13

#**6. Create a feature layer**

In [15]:
feature_columns = []
dataframe_copy = dataframe.copy()
dataframe_copy.pop('target')
def get_scal(feature):
  def minmax(x):
    mini = train[feature].min()
    maxi = train[feature].max()
    return (x - mini)/(maxi-mini)
  return(minmax)
  # def normalize_with_moments(x):
  #     axes=[0]
  #     epsilon=1e-8
  #     x_feat=tf.convert_to_tensor(train[feature], dtype=tf.float32)
  #     mean, variance = tf.nn.moments(x_feat, axes=axes)
  #     x_normed = (x - mean) / tf.sqrt(variance + epsilon) # epsilon to avoid dividing by zero
  #     return x_normed
  # return(normalize_with_moments)

for header in list(dataframe_copy.keys()):
    scal_input_fn = get_scal(header)
    feature_header=feature_column.numeric_column(header,normalizer_fn=scal_input_fn)
    feature_columns.append(feature_header)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
print(feature_columns)

[NumericColumn(key='f1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c740d0>), NumericColumn(key='f2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c74d90>), NumericColumn(key='f3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c57268>), NumericColumn(key='f4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c57048>), NumericColumn(key='f5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c571e0>), NumericColumn(key='f6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c57400>), NumericColumn(key='f7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function get_scal.<locals>.minmax at 0x7f3745c

#**7. Create, compile, and train the model**

In [16]:
checkpoint_filepath = './bestWeight/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    verbose=1,
    mode='auto',
    save_best_only=True, 
    period=1)

from tensorflow.keras import regularizers
import copy 
# train_ds_copy=copy(train_ds)
# val_ds_copy=copy(val_ds)
# kernel_regularizer=regularizers.l2(0.0001)
model = tf.keras.Sequential([
    feature_layer,
    layers.Dropout(.1),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
    layers.Dropout(.1),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
    layers.Dropout(.1),
    # keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    # layers.Dropout(.1),
    layers.Dense(30, activation = "softmax")
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.compile(
        optimizer='adam',
        loss=['sparse_categorical_crossentropy'],
        metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=30, 
          verbose=1,
          callbacks=[model_checkpoint_callback]
          )

Epoch 1/30
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
703/704 [============================>.] - ETA: 0s - loss: 0.2259 - accuracy: 0.9442WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'f1': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'f2': <tf.Tensor 'ExpandDims_136:0' shape=(None, 1) dtype=float64>, 'f3': <tf.Tensor 'ExpandDims_247:0' shape=(

In [17]:
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

#**8. Run test set on Model and export output.csv**

In [20]:
test_dataset_url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/21795/1405944/compressed/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1597545143&Signature=IwAqlRyUIP9vvWGeOGb8KS9teiVmDYW4FlL%2FOb7xTeXWafE%2BpUGfIa2lxCLHerW7hqV%2FHqFIzmUJp%2Fe%2F%2BLcF06iV8a0Ot5Ae0%2BYZfKeOFKPATrSnVr3UUTsOqlg1bfatpPEXklM%2F4xd9h8u4LrqWAjoegpJ2O%2F8gBnPPsElZzWU%2FcKQRQaMeF22ea1o81ePE9Tu6qgrtI2URHHQFvP61rzdrST9L1AqK9kb9zS6tyquqLeLT143W6jcM9URAYBChbudhZsOwprs74ts65Bib%2FY3HYsbnApOuF%2Fn89qB122wQpI%2FrVNyw%2F5nZ4pFsyfaBcMSB0oihtA8WgIIcBue4fA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv.zip'
test_csv_file = 'datasets/test.csv'

tf.keras.utils.get_file('test.csv.zip', test_dataset_url,
                        extract=True, cache_dir='.')
dataframe_test = pd.read_csv(test_csv_file)

33021952/33018136 [==============================] - 0s 0us/step


In [21]:
dataframe_test.head()

,# Id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f985,f986,f987,f988,f989,f990,f991,f992,f993,f994,f995,f996,f997,f998,f999,f1000,f1001,f1002,f1003,f1004,f1005,f1006,f1007,f1008,f1009,f1010,f1011,f1012,f1013,f1014,f1015,f1016,f1017,f1018,f1019,f1020,f1021,f1022,f1023,f1024
0,30001,1.548385,0.000000,1.821638,0.000000,1.765980,0.706634,0.000000,0.000000,0.118841,0.514384,0.280679,0.000000,0.000000,2.058144,0.000000,2.856107,1.773112,0.354395,2.027145,0.0,0.611815,0.000000,0.311365,0.0,0.244836,0.865879,0.0,0.000000,1.887619,1.863929,0.000000,2.367073,1.095520,0.323291,1.673315,0.000000,0.000000,2.670947,1.481659,...,0.472329,0.873205,0.000000,2.021482,0.341116,1.251182,2.243638,0.000000,0.070680,1.406880,0.994535,1.945908,0.086644,0.906351,0.164992,1.424353,1.263679,1.245625,1.410258,0.325073,0.918792,0.000000,1.296327,2.490774,0.000000,0.538412,0.813290,0.573764,0.129661,0.370074,1.242292,1.307992,1.270700,0.250358,1.034671,0.728463,1.292660,1.105686,0.299511,1.669667
1,30002,0.841867,0.000000,0.814518,0.326305,0.203243,0.278139,0.000000,0.793284,1.758104,0.764936,1.240908,2.072293,0.486326,1.691743,0.000000,1.929341,1.280567,0.401246,0.916930,0.0,0.047233,2.231684,1.196538,0.0,0.936710,1.263935,0.0,2.119725,0.611212,1.688470,1.660800,1.488553,0.000000,0.000000,0.937640,0.234417,0.000000,1.150364,0.315342,...,1.188820,0.000000,2.574032,0.447045,0.000000,0.000000,0.991384,0.627204,0.377995,0.622273,0.332809,0.721624,0.942563,0.000000,1.180423,3.147086,1.357030,0.000000,0.000000,1.452240,0.000000,1.305974,0.903263,0.000000,0.226647,0.931586,1.815364,1.001661,2.112926,0.222801,0.627378,0.323414,0.963698,1.160521,0.135187,0.953501,0.452462,1.937163,0.744784,0.000000
2,30003,0.513769,0.000000,2.244044,0.000000,0.000000,0.861955,0.084197,0.000000,0.000000,0.662475,0.439127,0.000000,0.000000,1.009468,0.159253,0.333241,1.093567,0.000000,1.919040,0.0,0.749443,0.000000,0.915152,0.0,0.170922,1.225322,0.0,0.000000,2.220622,0.725536,0.000000,2.556170,1.508196,1.450402,1.435504,0.000000,0.000000,1.289993,1.958031,...,0.947062,0.001745,1.391054,0.000000,0.000000,0.000000,0.000000,0.348993,1.884832,0.613375,0.000000,0.943784,1.996317,0.535431,0.022610,3.625532,1.777703,0.000000,0.000000,0.430689,0.000000,0.910035,0.893414,0.000000,0.250331,0.649465,0.939789,1.159956,1.272668,2.030895,0.521875,0.331770,0.343858,0.264370,0.157767,0.694368,0.479420,1.108407,0.564894,0.215731
3,30004,0.979345,0.185347,0.000000,0.120170,0.617745,0.177352,0.000000,1.903043,1.409005,1.477006,1.118432,2.070179,0.000000,1.083363,0.000000,0.950623,0.622709,0.000000,0.477551,0.0,0.302570,2.103658,1.840048,0.0,1.723238,0.000000,0.0,3.058941,0.023406,0.738614,1.535932,0.738068,0.000000,0.000000,1.802255,0.059761,0.000000,0.568850,0.833805,...,1.206887,0.000000,2.217909,0.000000,0.000000,0.422482,0.868588,0.153347,0.000000,1.473508,0.659295,0.000000,1.251002,0.303504,0.000000,0.942429,0.331096,0.689076,0.000000,1.272489,0.637485,0.473599,0.105639,0.524415,0.888827,0.000000,1.270760,0.000000,4.000865,0.000000,1.142519,0.000000,0.721612,1.002656,0.744457,1.262311,1.070073,0.086787,0.000000,1.113609
4,30005,0.000000,0.089834,0.000000,0.090698,0.000000,0.000000,0.000000,0.000000,0.000000,0.767373,0.466559,0.000000,0.000000,1.059852,0.000000,0.000000,0.000000,0.399487,0.000000,0.0,0.000000,0.757377,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.721347,0.000000,0.392565,0.000000,1.127471,0.000000,0.473168,0.000000,0.159748,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.485996,0.303347,0.399669,0.000000,1.120334,0.292400,0.882281,0.000000,0.212202,0.880501,0.000000,0.000000,0.209583,0.000000,0.409370,0.000000,0.000000,0.189278,0.000000,0.000000,0.000000,1.085576,0.832088,1.300438,0.000000,0.118793,0.386168,1.204944,0.682398,0.008352,0.774976,0.821177,0.000000,0.000000


In [22]:
# In the original dataset "4" indicates the pet was not adopted.
ouput_test={}
ouput_test['# Id'] = dataframe_test['# Id']

# Drop un-used columns.
dataframe_test = dataframe_test.drop(columns=['# Id'])

In [23]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset_test(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [24]:
dataframe_test_ds = df_to_dataset_test(dataframe_test, shuffle=False, batch_size=batch_size)

In [25]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(dataframe_test_ds)

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


In [26]:
print([np.argmax(predict) for predict in predictions])

[2, 3, 8, 3, 1, 8, 0, 5, 8, 7, 2, 7, 5, 2, 8, 0, 9, 6, 9, 0, 6, 7, 8, 3, 0, 9, 3, 8, 0, 1, 6, 1, 8, 5, 5, 7, 5, 4, 4, 2, 0, 4, 9, 0, 9, 4, 0, 5, 6, 7, 2, 8, 5, 6, 5, 4, 1, 9, 1, 2, 4, 5, 0, 0, 6, 8, 4, 5, 0, 9, 0, 2, 5, 7, 5, 2, 9, 7, 1, 9, 7, 8, 9, 3, 6, 3, 7, 4, 9, 8, 4, 7, 4, 8, 9, 4, 9, 1, 1, 4, 3, 9, 4, 7, 7, 0, 6, 8, 4, 8, 7, 0, 8, 9, 7, 2, 6, 2, 1, 9, 6, 5, 5, 2, 7, 8, 8, 7, 4, 9, 6, 2, 6, 9, 9, 1, 0, 6, 9, 8, 4, 9, 1, 9, 6, 0, 3, 3, 8, 8, 8, 1, 4, 4, 6, 3, 8, 1, 4, 8, 6, 0, 1, 7, 3, 9, 4, 7, 1, 3, 0, 8, 4, 0, 3, 9, 2, 8, 1, 0, 1, 6, 3, 3, 2, 9, 7, 4, 9, 8, 5, 7, 7, 6, 9, 9, 5, 1, 4, 4, 0, 8, 8, 8, 1, 4, 1, 1, 1, 9, 3, 2, 1, 3, 1, 0, 1, 7, 1, 6, 3, 3, 2, 6, 1, 6, 1, 6, 8, 1, 1, 9, 8, 3, 1, 8, 7, 9, 7, 8, 6, 0, 8, 5, 7, 1, 0, 6, 8, 3, 4, 5, 2, 0, 8, 4, 8, 3, 6, 7, 2, 2, 8, 1, 6, 4, 0, 3, 4, 1, 4, 1, 1, 4, 2, 0, 8, 2, 4, 8, 8, 2, 0, 7, 1, 9, 4, 8, 7, 2, 9, 4, 4, 1, 7, 3, 3, 1, 3, 2, 0, 2, 7, 6, 2, 9, 3, 7, 2, 4, 6, 8, 8, 1, 6, 3, 1, 0, 9, 5, 3, 4, 6, 3, 0, 1, 0, 1, 9, 7, 1, 7, 2, 

In [27]:
ouput_test['Category'] = [np.argmax(predict) for predict in predictions]
df = pd.DataFrame(ouput_test, columns= ['# Id', 'Category'])
df.to_csv('datasets/output.csv', index = False, header=True)


#**9. Save and reload model**

In [29]:
model.save('./model/model_save')
imported = tf.saved_model.load('./model/model_save')


Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
INFO:tensorflow:Assets written to: ./model/model_save/assets
